<a href="https://colab.research.google.com/github/HongJaeKwon/kimhae_fire_detection/blob/master/hjk_fire_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# /gdrive/My Drive/ (폴더명)

Mounted at /gdrive


In [0]:
import numpy as np
import pandas as pd
import os
import glob
import random
from tqdm import tqdm
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras import initializers
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input, Dropout, LeakyReLU, BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, Iterator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.python.keras.losses import mean_absolute_error, binary_crossentropy

In [176]:
train_df = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_train.csv')
val_df = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_validation.csv')
test_df= pd.read_csv('/gdrive/My Drive/김해화재/PJT002_test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
test_df['fr_yn']='U'

In [178]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(59199, 180)
(6898, 180)
(2957, 180)


In [179]:
all_df=pd.concat([train_df,val_df,test_df],axis=0)
all_df['fr_yn']=all_df['fr_yn'].apply(lambda x: 0 if x=='N' else 1 if x=='Y' else -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [180]:
all_df.head()

,ahsm_dstnc,bldng_ar,bldng_ar_prc,bldng_archtctr,bldng_cnt,bldng_cnt_in_50m,bldng_us,bldng_us_clssfctn,blk_dngrs_thng_mnfctr_yn,cctv_dstnc,cctv_in_100m,cltrl_hrtg_yn,dngrs_thng_yn,dt_of_athrztn,dt_of_fr,ele_engry_us_201401,ele_engry_us_201402,ele_engry_us_201403,ele_engry_us_201404,ele_engry_us_201405,ele_engry_us_201406,ele_engry_us_201407,ele_engry_us_201408,ele_engry_us_201409,ele_engry_us_201410,ele_engry_us_201411,ele_engry_us_201412,ele_engry_us_201501,ele_engry_us_201502,ele_engry_us_201503,ele_engry_us_201504,ele_engry_us_201505,ele_engry_us_201506,ele_engry_us_201507,ele_engry_us_201508,ele_engry_us_201509,ele_engry_us_201510,ele_engry_us_201511,ele_engry_us_201512,ele_engry_us_201601,...,gas_engry_us_201810,gas_engry_us_201811,gas_engry_us_201812,hm_cnt,hmdt,id,jmk,lnd_ar,lnd_us_sttn_nm,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,mlt_us_yn,no_tbc_zn_dstnc,prcpttn,rd_sd_nm,rgnl_ar_nm,rgnl_ar_nm2,sft_emrgnc_bll_dstnc,slf_fr_brgd_yn,tbc_rtl_str_dstnc,tmprtr,trgt_crtr,ttl_ar,ttl_dwn_flr,ttl_grnd_flr,us_yn,wnd_drctn,wnd_spd
0,11322,69.42,NaN,블록구조,3,0,단독주택,주거용,NaN,112,0,NaN,NaN,1977,2017-10-20 05:54:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17360.0,96.0,106536,대,0.0,단독,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,88,NaN,세로한면(불),자연녹지지역,지정되지않음,29231,NaN,1891,10.7,NaN,69.42,0.0,1.0,NaN,200.0,0.9
1,3369,46.29,122581.0,NaN,3,0,NaN,NaN,NaN,818,0,NaN,NaN,NaN,2018-09-30 08:26:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1791.0,74.0,86366,대,0.0,단독,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,7727,NaN,세로한면(가),계획관리지역,지정되지않음,20480,NaN,4533,19.5,NaN,46.29,0.0,1.0,NaN,20.0,0.5
2,12451,583.80,618105.0,철근콘크리트구조,1,14,공동주택,주거용,NaN,165,0,NaN,NaN,2.00011e+07,2016-10-30 14:57:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17285.0,21.0,107560,답,1446.0,연립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,72,NaN,중로한면,제2종일반주거지역,지정되지않음,31197,NaN,277,15.5,NaN,2516.76,0.0,5.0,NaN,90.0,2.0
3,407,48.92,719542.0,일반목구조,2,11,단독주택,주거용,NaN,131,0,NaN,NaN,1936,2016-06-14 05:23:58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7327.0,91.0,97550,대,0.0,단독,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,508,NaN,세로한면(불),제1종일반주거지역,지정되지않음,4270,NaN,438,20.6,NaN,48.92,0.0,1.0,NaN,0.0,0.4
4,12487,0.00,NaN,NaN,2,0,NaN,NaN,NaN,686,0,NaN,NaN,NaN,2018-04-22 05:38:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,17278.0,89.0,130501,임,0.0,자연림,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,707,NaN,세로한면(불),자연환경보전지역,지정되지않음,29778,NaN,1702,12.7,NaN,0.00,NaN,NaN,NaN,360.0,0.6


In [0]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

def hjk_feature_drop(df : pd.DataFrame, col_name :str, y_col='fr_yn',drop_rate=0.5):
    
    ms_rate=df[col_name].isnull().sum()/df.shape[0]
    print ('col_name : ',col_name)
    print ('dtype : ', df[col_name].dtype )
    print ('missing val : ', df[col_name].isnull().sum(), ms_rate*100,'%' )
    if(ms_rate>drop_rate):
        del df[col_name]
        print("deleted")
    print("")

def hjk_feature_missing(df : pd.DataFrame, col_name :str, y_col='fr_yn',strategy='mean'):
    imp = SimpleImputer(strategy=strategy)
            
    print ('col_name : ',col_name)
    print ('dtype : ', df[col_name].dtype )
    df[col_name]=imp.fit_transform((np.reshape(df[col_name].values,(-1,1))))
    print ('nan = ',strategy + '로 변환')
            
def hjk_feature_labeling(df : pd.DataFrame, col_name :str, y_col='fr_yn'):
    le = preprocessing.LabelEncoder()
    df[col_name]=le.fit_transform(df[col_name])

def hjk_feature_onehot(df : pd.DataFrame, col_name :str, y_col='fr_yn',unique_num=40):
    if(len(df[col_name].unique())<unique_num):
        enc = OneHotEncoder(handle_unknown='ignore')
        col=np.reshape(df[col_name].values,(-1,1))
        col=enc.fit_transform(col).toarray()
        col=pd.DataFrame(col,columns=list(col_name + "_" + str(i) for i in range(len(df[col_name].unique()))))
        df = df.reset_index(drop=True)
        df=pd.concat([df,col],axis = 1)
        del df[col_name]
    return df

In [182]:
all_f=all_df.columns.to_list()
print (len(all_f))

180


In [183]:
for f in all_f:
    hjk_feature_drop(all_df,f)


col_name :  ahsm_dstnc
dtype :  int64
missing val :  0 0.0 %

col_name :  bldng_ar
dtype :  float64
missing val :  0 0.0 %

col_name :  bldng_ar_prc
dtype :  float64
missing val :  26325 38.12233903901295 %

col_name :  bldng_archtctr
dtype :  object
missing val :  29224 42.32050279491413 %

col_name :  bldng_cnt
dtype :  int64
missing val :  0 0.0 %

col_name :  bldng_cnt_in_50m
dtype :  int64
missing val :  0 0.0 %

col_name :  bldng_us
dtype :  object
missing val :  29259 42.3711877660961 %

col_name :  bldng_us_clssfctn
dtype :  object
missing val :  31642 45.82211023257161 %

col_name :  blk_dngrs_thng_mnfctr_yn
dtype :  object
missing val :  57276 82.94378312624903 %
deleted

col_name :  cctv_dstnc
dtype :  int64
missing val :  0 0.0 %

col_name :  cctv_in_100m
dtype :  int64
missing val :  0 0.0 %

col_name :  cltrl_hrtg_yn
dtype :  object
missing val :  57276 82.94378312624903 %
deleted

col_name :  dngrs_thng_yn
dtype :  object
missing val :  57276 82.94378312624903 %
deleted


In [184]:
all_f=all_df.columns.to_list()
print (len(all_f))

157


In [0]:
all_f=all_df.columns.to_list()
cat_f=all_df.columns.to_list()
num_f=all_df.describe().columns.to_list()
for f in num_f:
    cat_f.remove(f)

In [186]:
for f in num_f:
    hjk_feature_missing(all_df,f)

col_name :  ahsm_dstnc
dtype :  int64
nan =  mean로 변환
col_name :  bldng_ar
dtype :  float64
nan =  mean로 변환
col_name :  bldng_ar_prc
dtype :  float64
nan =  mean로 변환
col_name :  bldng_cnt
dtype :  int64
nan =  mean로 변환
col_name :  bldng_cnt_in_50m
dtype :  int64
nan =  mean로 변환
col_name :  cctv_dstnc
dtype :  int64
nan =  mean로 변환
col_name :  cctv_in_100m
dtype :  int64
nan =  mean로 변환
col_name :  ele_engry_us_201401
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201402
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201403
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201404
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201405
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201406
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201407
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201408
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_201409
dtype :  float64
nan =  mean로 변환
col_name :  ele_engry_us_2

In [187]:
all_df['dt_of_athrztn']=all_df['dt_of_athrztn'].apply(lambda x : str(x)[:4])
hjk_feature_missing(all_df,'dt_of_athrztn')
all_df['dt_of_athrztn']=all_df['dt_of_athrztn'].astype('int64')

col_name :  dt_of_athrztn
dtype :  object
nan =  mean로 변환


In [0]:
all_df['dt_of_fr']=pd.to_datetime(all_df['dt_of_fr'])

In [0]:
all_df['month']=all_df['dt_of_fr'].dt.month
all_df['hour']=all_df['dt_of_fr'].dt.hour
all_df['weekday']=all_df['dt_of_fr'].dt.weekday

In [0]:
del all_df['dt_of_fr']

In [191]:
hjk_feature_missing(all_df,'month')
hjk_feature_missing(all_df,'hour')
hjk_feature_missing(all_df,'weekday')

col_name :  month
dtype :  int64
nan =  mean로 변환
col_name :  hour
dtype :  int64
nan =  mean로 변환
col_name :  weekday
dtype :  int64
nan =  mean로 변환


In [0]:
del all_df['emd_nm']

In [193]:
all_f=all_df.columns.to_list()
cat_f=all_df.columns.to_list()
num_f=all_df.describe().columns.to_list()
for f in num_f:
    cat_f.remove(f)
print (len(all_f))
print (len(cat_f))
print (len(num_f))

158
9
149


In [194]:
for f in cat_f:
    hjk_feature_missing(all_df,f,strategy='most_frequent')

col_name :  bldng_archtctr
dtype :  object
nan =  most_frequent로 변환
col_name :  bldng_us
dtype :  object
nan =  most_frequent로 변환
col_name :  bldng_us_clssfctn
dtype :  object
nan =  most_frequent로 변환
col_name :  jmk
dtype :  object
nan =  most_frequent로 변환
col_name :  lnd_us_sttn_nm
dtype :  object
nan =  most_frequent로 변환
col_name :  mlt_us_yn
dtype :  object
nan =  most_frequent로 변환
col_name :  rd_sd_nm
dtype :  object
nan =  most_frequent로 변환
col_name :  rgnl_ar_nm
dtype :  object
nan =  most_frequent로 변환
col_name :  rgnl_ar_nm2
dtype :  object
nan =  most_frequent로 변환


In [0]:
for f in cat_f:
    hjk_feature_labeling(all_df,f)

In [196]:
all_df.head()

,ahsm_dstnc,bldng_ar,bldng_ar_prc,bldng_archtctr,bldng_cnt,bldng_cnt_in_50m,bldng_us,bldng_us_clssfctn,cctv_dstnc,cctv_in_100m,dt_of_athrztn,ele_engry_us_201401,ele_engry_us_201402,ele_engry_us_201403,ele_engry_us_201404,ele_engry_us_201405,ele_engry_us_201406,ele_engry_us_201407,ele_engry_us_201408,ele_engry_us_201409,ele_engry_us_201410,ele_engry_us_201411,ele_engry_us_201412,ele_engry_us_201501,ele_engry_us_201502,ele_engry_us_201503,ele_engry_us_201504,ele_engry_us_201505,ele_engry_us_201506,ele_engry_us_201507,ele_engry_us_201508,ele_engry_us_201509,ele_engry_us_201510,ele_engry_us_201511,ele_engry_us_201512,ele_engry_us_201601,ele_engry_us_201602,ele_engry_us_201603,ele_engry_us_201604,ele_engry_us_201605,...,gas_engry_us_201707,gas_engry_us_201708,gas_engry_us_201709,gas_engry_us_201710,gas_engry_us_201711,gas_engry_us_201712,gas_engry_us_201801,gas_engry_us_201802,gas_engry_us_201803,gas_engry_us_201804,gas_engry_us_201805,gas_engry_us_201806,gas_engry_us_201807,gas_engry_us_201808,gas_engry_us_201809,gas_engry_us_201810,gas_engry_us_201811,gas_engry_us_201812,hm_cnt,hmdt,id,jmk,lnd_ar,lnd_us_sttn_nm,mlt_us_yn,no_tbc_zn_dstnc,rd_sd_nm,rgnl_ar_nm,rgnl_ar_nm2,sft_emrgnc_bll_dstnc,tbc_rtl_str_dstnc,tmprtr,ttl_ar,ttl_dwn_flr,ttl_grnd_flr,wnd_drctn,wnd_spd,month,hour,weekday
0,11322.0,69.42,700709.984647,8,3.0,0.0,9,6,112.0,0.0,1977,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,17360.0,96.0,106536.0,4,0.0,12,0,88.0,7,13,18,29231.0,1891.0,10.7,69.42,0.00000,1.000000,200.0,0.9,10.0,5.0,4.0
1,3369.0,46.29,122581.000000,15,3.0,0.0,9,6,818.0,0.0,1987,2878.88991,2865.160538,2552.847979,2388.883373,2225.330469,2252.572341,2442.149717,2657.321502,2563.455338,2643.000526,2928.598091,4663.481464,4585.374405,6355.080031,9659.541731,9236.401383,9144.604253,12477.35394,13898.785131,14733.092055,13305.3588,10758.016808,11956.698387,13144.567056,7772.944341,7168.379615,7253.667777,6508.237263,6080.202019,...,566890.795626,1.716001e+06,1.683419e+06,535357.100621,1.024853e+06,1.816973e+06,2.614779e+06,2.737553e+06,2.307607e+06,1.698565e+06,1.111269e+06,779296.519914,584146.9614,477265.736361,442930.823631,641739.501603,1.085383e+06,1.684235e+06,1791.0,74.0,86366.0,4,0.0,12,0,7727.0,6,1,18,20480.0,4533.0,19.5,46.29,0.00000,1.000000,20.0,0.5,9.0,8.0,6.0
2,12451.0,583.80,618105.000000,15,1.0,14.0,1,6,165.0,0.0,2000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,17285.0,21.0,107560.0,3,1446.0,25,0,72.0,11,17,18,31197.0,277.0,15.5,2516.76,0.00000,5.000000,90.0,2.0,10.0,14.0,6.0
3,407.0,48.92,719542.000000,10,2.0,11.0,9,6,131.0,0.0,1936,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,7327.0,91.0,97550.0,4,0.0,12,0,508.0,7,15,18,4270.0,438.0,20.6,48.92,0.00000,1.000000,0.0,0.4,6.0,5.0,1.0
4,12487.0,0.00,700709.984647,

In [197]:
all_f=all_df.columns.to_list()
cat_f=all_df.columns.to_list()
num_f=all_df.describe().columns.to_list()
for f in num_f:
    cat_f.remove(f)
print (len(all_f))
print (len(cat_f))
print (len(num_f))

158
0
158


In [0]:
all_f.remove('fr_yn')

In [199]:
l=[]
for f in all_f:
    if(len(all_df[f].unique())<40):
        l.append(f)
l

['bldng_archtctr',
 'bldng_us',
 'bldng_us_clssfctn',
 'cctv_in_100m',
 'fr_mn_cnt',
 'fr_wthr_fclt_in_100m',
 'jmk',
 'mlt_us_yn',
 'rd_sd_nm',
 'rgnl_ar_nm',
 'rgnl_ar_nm2',
 'ttl_dwn_flr',
 'wnd_drctn',
 'month',
 'hour',
 'weekday']

In [0]:
for f in all_f:
    all_df=hjk_feature_onehot(all_df,f)

In [201]:
all_df.head()

,ahsm_dstnc,bldng_ar,bldng_ar_prc,bldng_cnt,bldng_cnt_in_50m,cctv_dstnc,dt_of_athrztn,ele_engry_us_201401,ele_engry_us_201402,ele_engry_us_201403,ele_engry_us_201404,ele_engry_us_201405,ele_engry_us_201406,ele_engry_us_201407,ele_engry_us_201408,ele_engry_us_201409,ele_engry_us_201410,ele_engry_us_201411,ele_engry_us_201412,ele_engry_us_201501,ele_engry_us_201502,ele_engry_us_201503,ele_engry_us_201504,ele_engry_us_201505,ele_engry_us_201506,ele_engry_us_201507,ele_engry_us_201508,ele_engry_us_201509,ele_engry_us_201510,ele_engry_us_201511,ele_engry_us_201512,ele_engry_us_201601,ele_engry_us_201602,ele_engry_us_201603,ele_engry_us_201604,ele_engry_us_201605,ele_engry_us_201606,ele_engry_us_201607,ele_engry_us_201608,ele_engry_us_201609,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,11322.0,69.42,700709.984647,3.0,0.0,112.0,1977,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3369.0,46.29,122581.000000,3.0,0.0,818.0,1987,2878.88991,2865.160538,2552.847979,2388.883373,2225.330469,2252.572341,2442.149717,2657.321502,2563.455338,2643.000526,2928.598091,4663.481464,4585.374405,6355.080031,9659.541731,9236.401383,9144.604253,12477.35394,13898.785131,14733.092055,13305.3588,10758.016808,11956.698387,13144.567056,7772.944341,7168.379615,7253.667777,6508.237263,6080.202019,8536.832774,642004.097039,797868.975828,673044.970442,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,12451.0,583.80,618105.000000,1.0,14.0,165.0,2000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,407.0,48.92,719542.000000,2.0,11.0,131.0,1936,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,12487.0,0.00,700709.984647,2.0,0.0,686.0,1987,2878.88991,2865.160538,2552.847979,2388.883373,2225.330469,2252.572341,2442.149717,2657.321502,2563.455338,2643.000526,2928.598091,4663.481464,4585.374405,6355.080031,9659.541731,9236.401383,9144.604253,12477.35394,13898.785131,14733.092055,13305.3588,10758.016808,11956.698387,13144.567056,7772.944341,7168.379615,7253.667777,6508.237263,6080.202019,8536.832774,642004.097039,797868.975828,673044.970442,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [202]:
all_y=all_df['fr_yn'].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
all_f=all_df.columns.to_list()
all_f.remove('fr_yn')

In [0]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# all_df_minmax =scaler.fit_transform(all_df[all_f])

In [0]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
all_df_minmax =scaler.fit_transform(all_df[all_f])

In [0]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=200)
# all_df_pca=pca.fit_transform(all_df_minmax) 

In [0]:
# print(all_y.shape)
# print(all_df_pca.shape)
# print(pca.explained_variance_ratio_.sum())

In [0]:
all_y=np.reshape(all_y,(-1,1))
all_DF=pd.DataFrame(all_df_minmax)
all_DF_Y=pd.DataFrame(all_y,columns=['LABEL'])
all_DF=pd.concat([all_DF,all_DF_Y],axis = 1)

In [206]:
all_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,LABEL
0,0.415202,-0.203615,0.000000,0.333333,0.000000,-0.206897,-1.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002303,-0.002346,-0.002537,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.103019,-0.385594,-2.452791,0.333333,0.000000,0.807471,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.997697,0.997654,0.997463,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.488768,3.843355,-0.350463,-0.333333,4.666667,-0.130747,1.444444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002303,-0.002346,-0.002537,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-0.296024,-0.364902,0.079897,0.000000,3.666667,-0.179598,-5.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002303,-0.002346,-0.002537,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.491114,-0.749789,0.000000,0.000000,0.000000,0.617816,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.997697,0.997654,0.997463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
train_DF=all_DF[:train_df.shape[0]]
val_DF=all_DF[train_df.shape[0]:train_df.shape[0]+val_df.shape[0]]
test_DF=all_DF[all_DF['LABEL']==-1]

train_DF_Y=train_DF['LABEL']
val_DF_Y=val_DF['LABEL']
train_DF_normal=train_DF[train_DF['LABEL']==0]
train_DF_fire=train_DF[train_DF['LABEL']==1]

train_DF=train_DF.drop('LABEL', axis = 1)
val_DF=val_DF.drop('LABEL', axis = 1)
test_DF=test_DF.drop('LABEL', axis = 1)

train_DF_normal=train_DF_normal.drop('LABEL', axis = 1)
train_DF_fire=train_DF_fire.drop('LABEL', axis = 1)


In [208]:
f=train_DF_normal.columns.to_list()
for fe in f:
    print (fe,"정상 평균 : ", train_DF_normal[fe].mean())
    print (fe,"정상 표준편차 : ", train_DF_normal[fe].std())
    print (fe,"화재 평균 : ", train_DF_fire[fe].mean())
    print (fe,"화재 표준편차 : ", train_DF_fire[fe].std())
    print ("")

0 정상 평균 :  0.22951896042699846
0 정상 표준편차 :  0.7150981980439275
0 화재 평균 :  0.4823736938669327
0 화재 표준편차 :  0.8670859807961705

1 정상 평균 :  1.855767526168764
1 정상 표준편차 :  30.72259710308337
1 화재 평균 :  20.249468587181724
1 화재 표준편차 :  184.298216832964

2 정상 평균 :  -0.18586910622230987
2 정상 표준편차 :  1.677146664275412
2 화재 평균 :  -0.0086695695918802
2 화재 표준편차 :  1.5952570374173267

3 정상 평균 :  0.4325275180112348
3 정상 표준편차 :  1.868395833165459
3 화재 평균 :  0.8354446911322996
3 화재 표준편차 :  4.599279506440935

4 정상 평균 :  1.2463363211879284
4 정상 표준편차 :  2.7672641139841967
4 화재 평균 :  1.4426886073745155
4 화재 표준편차 :  3.0304817269681017

5 정상 평균 :  0.7459386988047221
5 정상 표준편차 :  1.9345105729528658
5 화재 평균 :  0.5784435472612406
5 화재 표준편차 :  1.821907642346736

6 정상 평균 :  -0.12336002138493342
6 정상 표준편차 :  7.931025847894528
6 화재 평균 :  0.4375081624657196
6 화재 표준편차 :  14.36994302660072

7 정상 평균 :  0.8308779822793002
7 정상 표준편차 :  44.778369297866625
7 화재 평균 :  1.3110033426007968
7 화재 표준편차 :  61.08434117452613

8 정상 

In [209]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=5)
decision_tree = decision_tree.fit(train_DF, train_DF_Y)
y_pred=decision_tree.predict(val_DF)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.7945781385908959
f1_score :  0.4930232558139535
precision_score :  0.45239658568614577
recall_score :  0.5416666666666666


In [210]:
from  sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=42)
rf_clf.fit(train_DF, train_DF_Y)
y_pred=rf_clf.predict(val_DF)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.8208176282980574
f1_score :  0.5075697211155378
precision_score :  0.5145395799676898
recall_score :  0.5007861635220126


In [211]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(train_DF, train_DF_Y)
y_pred = xgb_model.predict(val_DF)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.8079153377790664
f1_score :  0.5176556243174372
precision_score :  0.48203389830508475
recall_score :  0.5589622641509434


In [0]:
class HJK_NN:
    def __init__(self,batch_size=128, input_size=436):
        self.batch_size=batch_size
        self.input_size=input_size
        self.lr = 0.001
        self.dropout_rate = 0.3
        self.val_ratio = 0.3
        self.buld_model()
    
    def buld_model(self):
        self.model = Sequential()
        
        self.model.add(Dense(100,activation='relu',input_shape=(self.input_size,)))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))

        self.model.add(Dense(100,activation='relu'))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))

        self.model.add(Dense(100,activation='relu'))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))

        self.model.add(Dense(50,activation='relu'))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))

        self.model.add(Dense(20,activation='relu'))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
        
        self.model.add(Dense(10,activation='relu'))
        self.model.add(Dropout(rate=self.dropout_rate))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))

        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(loss=self.hjk_loss, metrics=['acc',self.f1_m,self.precision_m, self.recall_m],optimizer=Adam(self.lr))

    def hjk_train(self,trian_X,train_Y,epochs=20):
        self.model.fit(trian_X,train_Y,batch_size=self.batch_size,epochs=epochs,validation_split=self.val_ratio)
    
    def hjk_predict(self,trian_X):
        return self.model.predict(trian_X)
           
    def recall_m(self,y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self,y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self,y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    def hjk_loss(self,y_true,y_pred):
        bce = 5*y_true * K.log(y_pred + K.epsilon())
        bce += (1 - y_true) * K.log(1 - y_pred + K.epsilon())
        return -bce

    def f1_loss(self,y_true, y_pred):
        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())

        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = K.clip(f1,K.epsilon(),1-K.epsilon())
        return 1 - K.mean(f1)     

In [0]:
hjk=HJK_NN()

In [231]:
hjk.hjk_train(train_DF.as_matrix(),train_DF_Y.as_matrix(),20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 41439 samples, validate on 17760 samples
Epoch 1/20
41439/41439 [==============================] - 14s 340us/sample - loss: 0.8580 - acc: 0.6060 - f1_m: 0.3157 - precision_m: 0.2000 - recall_m: 0.8007 - val_loss: 0.9847 - val_acc: 0.8212 - val_f1_m: 0.2646 - val_precision_m: 0.4274 - val_recall_m: 0.1984
Epoch 2/20
41439/41439 [==============================] - 6s 138us/sample - loss: 0.6573 - acc: 0.7284 - f1_m: 0.4072 - precision_m: 0.2724 - recall_m: 0.8484 - val_loss: 1.1298 - val_acc: 0.8318 - val_f1_m: 0.1040 - val_precision_m: 0.4676 - val_recall_m: 0.0600
Epoch 3/20
41439/41439 [==============================] - 6s 136us/sample - loss: 0.6026 - acc: 0.7484 - f1_m: 0.4270 - precision_m: 0.2897 - recall_m: 0.8575 - val_loss: 1.2746 - val_acc: 0.8396 - val_f1_m: 0.1516 - val_precision_m: 0.6276 - val_recall_m: 0.0882
Epoch 4/20
41439/41439 [==============================] - 6s 143us/sample - loss: 0.5808 - acc: 0.7540 - f1_m: 0.4394 - precision_m: 0.2988 - recall_m: 0.877

In [232]:
hjk.model.evaluate(val_DF.as_matrix(),val_DF_Y.as_matrix())

 960/6898 [===>..........................] - ETA: 1s - loss: 1.2577 - acc: 0.8500 - f1_m: 0.1920 - precision_m: 0.4639 - recall_m: 0.1325

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


6898/6898 [==============================] - 1s 159us/sample - loss: 1.2385 - acc: 0.8372 - f1_m: 0.2314 - precision_m: 0.5459 - recall_m: 0.1541


[1.2385081457863618, 0.8371992, 0.23143047, 0.5459105, 0.1541095]

In [0]:
real_list=np.reshape(hjk.hjk_predict(train_DF.as_matrix()),(-1,1))
dt_list=np.reshape(xgb_model.predict_proba(train_DF)[:,1],(-1,1))
rf_list=np.reshape(rf_clf.predict_proba(train_DF)[:,1],(-1,1))
print(real_list.shape)
print(dt_list.shape)
print(rf_list.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(59199, 1)
(59199, 1)
(59199, 1)


In [0]:
t_X=np.hstack((real_list,dt_list,rf_list))

In [0]:
stack_model = Sequential()
stack_model.add(Dense(1,activation='sigmoid',input_shape=(3,)))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def hjk_loss(y_true,y_pred):
    bce = 5*y_true * K.log(y_pred + K.epsilon())
    bce += (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    return -bce   

stack_model.compile(loss=hjk_loss,metrics=['acc',f1_m,precision_m, recall_m],optimizer=Adam(0.01))

In [0]:
stack_model.fit(t_X,train_DF_Y.as_matrix(),batch_size=100,epochs=10,validation_split=0.3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 41439 samples, validate on 17760 samples
Epoch 1/10
41439/41439 [==============================] - 2s 39us/sample - loss: 0.4938 - acc: 0.9481 - f1_m: 0.8195 - precision_m: 0.7221 - recall_m: 0.9749 - val_loss: 0.3843 - val_acc: 0.9349 - val_f1_m: 0.8318 - val_precision_m: 0.7354 - val_recall_m: 0.9647
Epoch 2/10
41439/41439 [==============================] - 1s 35us/sample - loss: 0.1980 - acc: 0.9839 - f1_m: 0.9285 - precision_m: 0.8768 - recall_m: 0.9938 - val_loss: 0.2285 - val_acc: 0.9665 - val_f1_m: 0.9057 - val_precision_m: 0.8370 - val_recall_m: 0.9925
Epoch 3/10
41439/41439 [==============================] - 2s 37us/sample - loss: 0.1176 - acc: 0.9921 - f1_m: 0.9653 - precision_m: 0.9362 - recall_m: 0.9995 - val_loss: 0.1486 - val_acc: 0.9838 - val_f1_m: 0.9535 - val_precision_m: 0.9142 - val_recall_m: 0.9985
Epoch 4/10
41439/41439 [==============================] - 2s 36us/sample - loss: 0.0771 - acc: 0.9958 - f1_m: 0.9811 - precision_m: 0.9647 - recall_m: 0.9998 - v

In [0]:
real_list=np.reshape(hjk.hjk_predict(val_DF.as_matrix()),(-1,1))
dt_list=np.reshape(xgb_model.predict_proba(val_DF)[:,1],(-1,1))
rf_list=np.reshape(rf_clf.predict_proba(val_DF)[:,1],(-1,1))
print(real_list.shape)
print(dt_list.shape)
print(rf_list.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(6898, 1)
(6898, 1)
(6898, 1)


In [0]:
v_X=np.hstack((real_list,dt_list,rf_list))

In [0]:
y_pred=v_X.mean(axis=1)
y_pred=np.round(y_pred)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.7441287329660772
f1_score :  0.49091433516008076
precision_score :  0.3876993166287016
recall_score :  0.6690251572327044


In [0]:
stack_model.evaluate(v_X,val_DF_Y.as_matrix())

2880/6898 [===========>..................] - ETA: 0s - loss: 1.1179 - acc: 0.7247 - f1_m: 0.4580 - precision_m: 0.3581 - recall_m: 0.7146

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


6898/6898 [==============================] - 0s 54us/sample - loss: 1.1515 - acc: 0.7259 - f1_m: 0.4597 - precision_m: 0.3648 - recall_m: 0.6837


[1.1514912743124626, 0.72586256, 0.45969123, 0.36482933, 0.68372035]

In [0]:
real_list=np.reshape(hjk.hjk_predict(test_DF.as_matrix()),(-1,1))
dt_list=np.reshape(xgb_model.predict_proba(test_DF)[:,1],(-1,1))
rf_list=np.reshape(rf_clf.predict_proba(test_DF)[:,1],(-1,1))
print(real_list.shape)
print(dt_list.shape)
print(rf_list.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(2957, 1)
(2957, 1)
(2957, 1)


In [0]:
# test_X=np.hstack((real_list,dt_list,rf_list))
y_pred = xgb_model.predict(test_DF)
# y_pred=np.reshape(hjk.hjk_predict(test_DF.as_matrix()),(-1,1))
# y_pred=np.round(y_pred)
sub = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_submission.csv')

sub['fr_yn'] = y_pred
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.to_csv('/gdrive/My Drive/김해화재/PJT002_submission_hjk7.csv', index=False)